##### 포즈 landmarks 좌표 소켓 서버 전송

In [ ]:
import mediapipe as mp
import cv2
import socket

# TCP 서버 설정
def send_pose_landmarks(host='127.0.0.1', port=6521):
    # 소켓 객체 생성
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        # 서버에 연결
        client_socket.connect((host, port))
        print(f"서버에 연결되었습니다. {host}:{port}")

        # MediaPipe Pose 설정
        pose = mp.solutions.pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            smooth_landmarks=True,
            enable_segmentation=False,  # 세그멘테이션 기능 비활성화
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)

        # 그리기 함수 초기화
        mp_drawing = mp.solutions.drawing_utils

        # 웹캠 캡처
        cap = cv2.VideoCapture(0)

        while True:
            # 이미지 읽기
            ret, image = cap.read()
            if not ret:
                break

            # 이미지 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 포즈 추정
            results = pose.process(image)

            # 추정된 포즈 정보 확인
            if results.pose_landmarks:
                # 포즈 랜드마크를 리스트로 변환하여 문자열로 만듦
                pose_landmarks_str = ';'.join(
                    [f"{lm.x},{lm.y},{lm.z}" for lm in results.pose_landmarks.landmark])

                # 포즈 랜드마크 문자열 출력
                print(pose_landmarks_str)

                # 포즈 랜드마크 문자열을 서버로 전송
                client_socket.sendall(pose_landmarks_str.encode())

            # 이미지 출력
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
            cv2.imshow('MediaPipe Pose', image)

            # 키 입력 처리
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

        # 자원 해제
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    send_pose_landmarks()

서버에 연결되었습니다. 127.0.0.1:6521
0.7706181406974792,0.8441368341445923,-1.7363622188568115;0.7889387607574463,0.7535703182220459,-1.6657698154449463;0.8081457018852234,0.7491003274917603,-1.6662323474884033;0.8271252512931824,0.7450616359710693,-1.6666982173919678;0.7182265520095825,0.7673830389976501,-1.6796573400497437;0.689236581325531,0.7726061344146729,-1.6793930530548096;0.6623008251190186,0.7772617340087891,-1.679932951927185;0.8533622622489929,0.7629802227020264,-1.074537992477417;0.6223670244216919,0.8170145750045776,-1.114654302597046;0.8092592358589172,0.9163758754730225,-1.4974219799041748;0.7339165806770325,0.9266382455825806,-1.510142207145691;0.9856206178665161,1.0283074378967285,-0.6298125386238098;0.5186713933944702,1.1824092864990234,-0.6831579208374023;1.1409330368041992,1.32522451877594,-0.6326403617858887;0.47684651613235474,1.5726101398468018,-0.5446650385856628;1.0580307245254517,1.4312046766281128,-1.0227833986282349;0.5484361052513123,1.6853201389312744,-0.855193972